In [240]:
import getpass
import os
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import BaseMessage, ToolMessage, SystemMessage
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph, START, END, MessagesState
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from dotenv import load_dotenv
from typing import List, Union
import logging

In [241]:
load_dotenv()
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [242]:
class AgentState(TypedDict):
    messages: Annotated[List[Union[HumanMessage, AIMessage, ToolMessage]], add_messages]

In [243]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
search = DuckDuckGoSearchRun()
logger.info(f"description = {search.description}")
tools = [search]
llm_with_tools = model.bind_tools(tools)

INFO:__main__:description = A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.


In [244]:
tool_by_name = {tool.name: tool for tool in tools}
tool_by_name

{'duckduckgo_search': DuckDuckGoSearchRun(api_wrapper=DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='auto', source='text'))}

In [245]:
result = search.invoke("top 10 horror movies to watch?")
print(result)

/opt/miniconda3/envs/berkeley_ai/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
INFO:primp:response: https://www.bing.com/search?q=top+10+horror+movies+to+watch%3F&filters=ex1%3A%22ez5_19972_20337%22 200
DEBUG:duckduckgo_search.DDGS:_get_url() https://www.bing.com/search?q=top+10+horror+movies+to+watch%3F&filters=ex1%3A%22ez5_19972_20337%22 200


No good DuckDuckGo Search Result was found


In [246]:
def call_model(state: AgentState, config: RunnableConfig):
    system_prompt = SystemMessage("You are a helpful AI assistant that searches the web."
        "Use the search tool duckduckgo_search to query the web for relevant information."
    )
    # system_prompt = SystemMessage("")
    response = llm_with_tools.invoke([system_prompt] + list(state["messages"]), config=config)
    logger.debug(f"Model response: {response}")
    return {"messages": [response]}


In [247]:
def tool_node(state: AgentState, config: RunnableConfig):
   logger.info(f"Invoking tools for {state['messages'][-1].tool_calls}")
   outputs = []
   for tool_call in state["messages"][-1].tool_calls:
      logger.info(f"tool call: {tool_call}")
      tool_result = tool_by_name[tool_call["name"]].invoke(tool_call["args"], config=config)
      logger.info(f"Tool {tool_call["name"]} returned {tool_result}")
      outputs.append(ToolMessage(
         content=str(tool_result),
         name=tool_call["name"],
         tool_call_id=tool_call["id"]
      ))

   return {"messages": outputs}

In [248]:
def should_continue(state: AgentState) -> bool:
    messages = state["messages"]
    last_message = messages[-1]
    if not last_message.tool_calls:
        logger.info("No tool calls, ending")
        return "end"
    else:
        return "continue"

In [249]:
workflow = StateGraph(AgentState)
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, {"continue": "tools", "end": END})
workflow.add_edge("tools", "agent")

In [250]:
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "thread_1"}}

In [ ]:
response = graph.invoke({
    "messages": [HumanMessage(content="search top 10 horror movies to watch?")]
}, config=config)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'idempotency_key': 'stainless-python-retry-bf2bf7a8-5feb-46ea-b69e-53f432076bfc', 'json_data': {'messages': [{'content': 'You are a helpful AI assistant that searches the web.Use the search tool duckduckgo_search to query the web for relevant information.', 'role': 'system'}, {'content': 'search the capital of Japan', 'role': 'user'}], 'model': 'gpt-4o-mini', 'stream': False, 'temperature': 0.0, 'tools': [{'type': 'function', 'function': {'name': 'duckduckgo_search', 'description': 'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}]}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:

In [252]:
print(response["messages"][-1].content)

The capital of Japan is Tokyo. It became the capital in 1869, following the end of the Boshin War, which restored direct Imperial rule after the defeat of the Tokugawa Shogunate. Before Tokyo, the capital was Kyoto.
